In [1]:
import pandas as pd
import datetime
import os
import itertools

In [ ]:
traductor_sii = {'33': 'FACTURAS ELECTRONICAS', '34': 'FACTURAS EXENTAS', '61': 'NOTAS DE CREDITO'}
diccionario_sii = {'FACTURAS ELECTRONICAS': [], 'FACTURAS EXENTAS': [], 'NOTAS DE CREDITO': []}
diccionario_dfs = {'SII': diccionario_sii, 'ACEPTA': [], 'PRESUPUESTO': [], 'SCI': []}

for nombre_archivo in os.listdir():
    if 'DTE_RECIBIDOS' in nombre_archivo:
        identificador = nombre_archivo.split('.')[0].split('_')[-1]
        tipo_archivo = traductor_sii[identificador]
        diccionario_sii[tipo_archivo].append(pd.read_csv(nombre_archivo, delimiter = ';'))

    elif 'Documentos recepcion' in nombre_archivo:
        diccionario_dfs['SCI'] = pd.read_excel(nombre_archivo)






In [2]:
df_sii_facturas_electronicas = pd.read_csv('DTE_RECIBIDOS_61608402_202209_33.csv', delimiter = ';')
df_sii_exentas = pd.read_csv('DTE_RECIBIDOS_61608402_202209_34.csv', delimiter = ';')
df_sii_notas_de_credito = pd.read_csv('DTE_RECIBIDOS_61608402_202209_61.csv', delimiter = ';')

df_acepta = pd.read_excel('acepta_2022.xlsx')


df_presupuesto = pd.read_excel('presupuesto 06.09.2022.xlsx')
df_sci = pd.read_excel('Documento recepcion (36).xlsx')






In [10]:
df_sii_exentas = pd.merge(df_sii, df_exentas, how = 'left', left_index = True, right_index = True)
df_sii_exentas_presupuesto = pd.merge(df_sii_exentas, df_presupuesto, how = 'left', left_index = True, right_index = True)
df_sii_exentas_presupuesto_sci = pd.merge(df_sii_exentas_presupuesto, df_sci, how = 'left', left_index = True, right_index = True)
df_sii_exentas_presupuesto_sci_acepta = pd.merge(df_sii_exentas_presupuesto_sci, df_acepta, how = 'left', left_index = True, right_index = True)

df_sii_exentas_presupuesto_sci_acepta = df_sii_exentas_presupuesto_sci_acepta[~df_sii_exentas_presupuesto_sci_acepta.index.duplicated(keep = 'first')] 

df_sii_exentas_presupuesto_sci_acepta['tiempo_diferencia'] = pd.to_datetime('today') - df_sii_exentas_presupuesto_sci_acepta['Fecha Docto. SII']
df_sii_exentas_presupuesto_sci_acepta['esta_al_dia'] = df_sii_exentas_presupuesto_sci_acepta['tiempo_diferencia'] <= datetime.timedelta(8)

columnas_a_ocupar = ['Nro. SII', 'RUT Emisor SII', 'razon_social_emisor ACEPTA', 'folio_oc ACEPTA', 'Folio SII', 'Fecha Docto. SII', 'Monto Neto SII', 
                     'Monto Exento SII', 'Monto IVA SII', 'Monto Total SII', 'Fecha Recep. SII', 'Ubic. PRESUPUESTO',
                     'tarea_actual ACEPTA', 'estado_devengo ACEPTA', 'fecha_ingreso_rc ACEPTA', 'estado_nar ACEPTA', 'Motivo SCI', 'tiempo_diferencia', 'esta_al_dia', 'Evento Receptor SII']

df_columnas_utiles = df_sii_exentas_presupuesto_sci_acepta[columnas_a_ocupar]


# df_sii_exentas_presupuesto_sci_acepta.to_excel('MATRIZ_FINAL_COMPLETA.xlsx')
fecha_actual = str(pd.to_datetime('today')).split(' ')[0]
nombre_archivo = f'PLANILLA DE CONTROL AL {fecha_actual}.xlsx'

if nombre_archivo in os.listdir():
    with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'a', if_sheet_exists = 'overlay') as writer:
        df_columnas_utiles.to_excel(writer)
else:
    df_columnas_utiles['Observaciones'] = None
    with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'w') as writer:
        df_columnas_utiles.to_excel(writer)

In [ ]:
class GeneradorPlanillaFinanzas:
    def __init__(self):
        self.correr_programa()

    def correr_programa(self):
        diccionario_dfs = self.identificar_tipo_de_archivo_y_cargar_dfs()
        dfs_limpias = self.limpiar_dfs(diccionario_dfs)
        df_final = self.unir_dfs(dfs_limpias)
    
    def limpiar_dfs(self, diccionario_dfs):
        for nombre_tabla, df in diccionario_dfs.items():
            if nombre_tabla == 'ACEPTA':
                df.rename(columns = {'emisor': 'RUT Emisor', 'folio': 'Folio'}, inplace = True)
            
            elif nombre_tabla == 'PRESUPUESTO':
                df.rename(columns = {'Rut': 'RUT Emisor', 'Folio': 'Folio_abreviado', 'NºDoc.': 'Folio'}, inplace = True)

            elif nombre_tabla == 'SCI':
                df.rename(columns = {'Rut Proveedor': 'RUT Emisor', 'Numero Documento': 'Folio'}, inplace = True)
            
            df.columns = df.columns + f' {nombre_tabla}'
            df['llave_id'] = df[f'RUT Emisor {nombre_tabla}'].astype(str) + df[f'Folio {nombre_tabla}'].astype(str)
            df.set_index('llave_id', drop = True, inplace = True)
        
        return diccionario_dfs
    
    def unir_dfs(self, dfs_limpias):
        lista_dfs_secuenciales = list(itertools.chain(*dfs_limpias.values()))
        df_izquierda = lista_dfs_secuenciales[0]

        for df_derecha in lista_dfs_secuenciales[1:]:
            df_izquierda = pd.merge(df_izquierda, df_derecha, how = 'left', left_index = True, right_index = True)
        
        df_izquierda = df_izquierda[~df_izquierda.index.duplicated(keep = 'first')]

        df_izquierda['tiempo_diferencia'] = pd.to_datetime('today') - df_izquierda['Fecha Docto. SII']
        df_izquierda['esta_al_dia'] = df_izquierda['tiempo_diferencia'] <= datetime.timedelta(8)

        columnas_a_ocupar = ['Nro. SII', 'RUT Emisor SII', 'razon_social_emisor ACEPTA', 'folio_oc ACEPTA', 'Folio SII', 'Fecha Docto. SII', 'Monto Neto SII', 
                     'Monto Exento SII', 'Monto IVA SII', 'Monto Total SII', 'Fecha Recep. SII', 'Ubic. PRESUPUESTO',
                     'tarea_actual ACEPTA', 'estado_devengo ACEPTA', 'fecha_ingreso_rc ACEPTA', 'estado_nar ACEPTA', 'Motivo SCI', 'tiempo_diferencia', 'esta_al_dia', 'Evento Receptor SII']
        
        df_util = df_izquierda[columnas_a_ocupar]

        return df_izquierda, df_util
        

        
        
